# KernelPyLogit using KFold Cross-validation

The purpose of this Jupyter notebook is to demonstrate how the KFold Cross-val function can be used to estimate a Random Utility Model (RUM) and a Kernel Logistic Regression (KLR) model. It is assumed that the user has already read the 'Main PyKernelLogit Example' notebook.

The dataset selected for this example is the 'Mode' dataset from the 'mlogit' package for R programming language (https://cran.r-project.org/web/packages/mlogit/mlogit.pdf). This dataset was elaborated by a project team in 'Discrete choice methods with simulation' (Train, 2003) and contains data about the mode choice of $453$ commuters. Each commuter has four available travel modes for their trips to work in a choice set $C=\{ \hbox{}bus, car, carpool, rail  \}$. The time and cost of travel for each mode were determined for each commuter. The attributes of the alternative $i \in C$ for the commuter $n$  are $x_{in}=(x_{in,cost}, x_{in,time})$ .


The first step is to load the different packages that will be used in this example.

In [1]:
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import PyKernelLogit as pkl            # For the estimation of the MNL and KLR models.

## 1. Loading and preprocessing the data

### 1.1. Load the data
The first step to estimate a model using PyKernelLogit is loading the dataset using a pandas dataframe.

In [2]:
# Load the Mode dataset from a .csv file. Nota that all the elements are delimited by a ',' character.
mode_choice = pd.read_csv("../data/ModeChoice.csv", sep=',')

# Show the first 5 rows of the data
mode_choice.head().T

,0,1,2,3,4
choice,car,rail,car,car,car
cost.car,1.50701,6.057,5.79468,1.86914,2.49895
cost.carpool,2.33561,2.89692,2.13745,2.57243,1.72201
cost.bus,1.80051,2.23713,2.57638,1.90352,2.686
cost.rail,2.35892,1.85545,2.74748,2.26828,2.97387
time.car,18.5032,31.3111,22.5474,26.0903,4.69914
time.carpool,26.3382,34.257,23.2552,29.896,12.4141
time.bus,20.8678,67.1819,63.3091,19.7527,43.092
time.rail,30.0335,60.2931,49.1716,13.4727,39.7433


### 1.2. Convert the dataset to long format

In [3]:
# Look at the columns of the Mode dataset
mode_choice.columns

Index(['choice', 'cost.car', 'cost.carpool', 'cost.bus', 'cost.rail',
       'time.car', 'time.carpool', 'time.bus', 'time.rail'],
      dtype='object')

In [4]:
# Convert choice from categorical variable to numeric one
mode_choice['choice'] = mode_choice['choice'].map({'car' : 1, 'carpool' : 2 , 'bus' : 3, 'rail' : 4})

# Create the list of individual specific variables
ind_variables = [] # There are no indivicual specific variables in this dataset

# Variables that vary across some or all alternatives, for a given individual
alt_varying_variables = {u'cost': dict([(1, 'cost.car'),
                                        (2, 'cost.carpool'),
                                        (3, 'cost.bus'),
                                        (4, 'cost.rail')]),
                         u'time': dict([(1, 'time.car'),
                                        (2, 'time.carpool'),
                                        (3, 'time.bus'),
                                        (4, 'time.rail')])
                        }

### Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset. If the dataset does not contains any
# availability columns (such as in the Mode dataset), then the 
# keyword 'available_for_all' can be used to specify that an alternative
# is availables for all individuals.
availability_variables = {1: 'available_for_all',
                          2: 'available_for_all', 
                          3: 'available_for_all',
                          4: 'available_for_all'}

### Determine the columns for: alternative ids, the observation ids and the choice

# The 'custom_alt_id' is the name of a column to be created in the long format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
mode_choice[obs_id_column] = np.arange(mode_choice.shape[0], dtype=int) + 1

# Create a variable recording the choice column
choice_column = "choice"

In [5]:
# Perform the conversion to long format
long_mode_choice = pkl.convert_wide_to_long(mode_choice, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)

In [6]:
# Look at the resulting long format dataframe
long_mode_choice.head(10).T

,0,1,2,3,4,5,6,7,8,9
custom_id,1.00000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000
mode_id,1.00000,2.000000,3.000000,4.000000,1.000000,2.000000,3.000000,4.000000,1.000000,2.000000
choice,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
cost,1.50701,2.335612,1.800512,2.358920,6.056998,2.896919,2.237128,1.855451,5.794677,2.137454
time,18.50320,26.338233,20.867794,30.033469,31.311107,34.256956,67.181889,60.293126,22.547429,23.255171


## 2. Estimation of a RUM using KFold Cross-validation

In Machine Learning (ML) it is recommended to perform a cross-validation, which is a resampling procedure used to estimate the skill of ML models on a limited data sample. The most commonly cross-validation technique is K-fold cross-validation. This procedure has a single parameter called $K$ which contains the number of subsets, called folds, in which a given data sample are divided. If $K = 10$, for instance, then the procedure is named 10-fold cross-validation. Then, a test subset is generated picking a different fold for every iteration and the other $K-1$ folds are used as the training subset. Then, the model is estimated using this training subset and tested on the test subset for each of the $K$ iterations. The final results are commonly computed as the average of the individual results for each iteration.

PyKernelLogit provides a function named KFold_cross_val which implements KFold Cross-validation. This function takes the following input arguments:
* long_data. The pandas dataframe in long format which is going to be used by the cross-validator.

* obs_id_col. The column from the wide dataset that contains the ID of each observation.

* n_splits. The number of folds ($K$).

* shuffle. A boolean parameter that determines whether or not the data must be shuffled before splitting it into batches.

* random_state. An integer value which is used as the seed for the random number generator.

The KFold_cross_val function uses a Python yield statement that creates a generator function which returns an iterator known as a generator iterator. The body of the generator function is executed by calling the generator's next() method repeatedly until it raises an exception. The main advantage of the yield statement is that it simplifies the application of the K-fold cross-validation, because the KFold_cross_val function can be called inside a for loop. Then, at each iteration of the loop, the corresponding training and test set are computed by this generator iterator.

The next code estimates and evaluates a RUM using 10-Fold Cross-validation. The model specification used is the same as in the 'Main PyKernelLogit Example' notebook.

In [7]:
# Create an empty basic_specification and basic_names ordered dictionary
basic_specification = OrderedDict()
basic_names = OrderedDict()

# Model specification
basic_specification["intercept"] = [2, 3, 4]
basic_names["intercept"] = ['ASC CarPool',
                            'ASC Bus',
                            'ASC Rail']

basic_specification["cost"] = [[1, 2, 3, 4]]
basic_names["cost"] = ['Travel cost (dollars)']

basic_specification["time"] = [[1, 2, 3, 4]]
basic_names["time"] = ['Travel time (minutes)']

In [8]:
fold = 0
precision = 0
recall = 0
fscore = 0

n_splits = 10

for train_set, test_set in pkl.KFold_cross_val(long_mode_choice,
                                               obs_id_column,
                                               n_splits,
                                               shuffle=False,
                                               random_state=1):
    
    fold += 1
    print("--- Set %d ---" % fold)
    
    # Estimate the multinomial logit model (MNL)
    linear_model = pkl.create_choice_model(data=train_set,
                                            alt_id_col=custom_alt_id,
                                            obs_id_col=obs_id_column,
                                            choice_col=choice_column,
                                            specification=basic_specification,
                                            model_type="MNL",
                                            names=basic_names)
    
    # Specify the initial values and method for the optimization.
    linear_model.fit_mle(np.zeros(5))
    print("Rho squared: %f" % linear_model.rho_squared)
    
    (precision_fold, recall_fold, fscore_fold) = linear_model.precision_recall_fscore(test_set, "micro", beta = 1)

    print("Precision: %f, Recall: %f, FScore: %f" % (precision_fold, recall_fold, fscore_fold))
    
    precision += precision_fold
    recall += recall_fold
    fscore += fscore_fold
    
print("\n --------------------- Final Results ---------------------")
print("Precision: %f, Recall: %f, FScore: %f" % (precision/n_splits, recall/n_splits, fscore/n_splits))

--- Set 1 ---
Log-likelihood at zero: -564.2218
Initial Log-likelihood: -564.2218
Estimation Time for Point Estimation: 0.01 seconds.
Final log-likelihood: -319.1546
Rho squared: 0.434345
Precision: 0.739130, Recall: 0.739130, FScore: 0.739130
--- Set 2 ---
Log-likelihood at zero: -564.2218
Initial Log-likelihood: -564.2218
Estimation Time for Point Estimation: 0.01 seconds.
Final log-likelihood: -316.4052
Rho squared: 0.439218
Precision: 0.608696, Recall: 0.608696, FScore: 0.608696
--- Set 3 ---
Log-likelihood at zero: -564.2218
Initial Log-likelihood: -564.2218
Estimation Time for Point Estimation: 0.01 seconds.
Final log-likelihood: -322.1470
Rho squared: 0.429042
Precision: 0.695652, Recall: 0.695652, FScore: 0.695652
--- Set 4 ---
Log-likelihood at zero: -565.6081
Initial Log-likelihood: -565.6081
Estimation Time for Point Estimation: 0.01 seconds.
Final log-likelihood: -322.0688
Rho squared: 0.430580
Precision: 0.688889, Recall: 0.688889, FScore: 0.688889
--- Set 5 ---
Log-likeli

## 3. Estimation of a KLR model using KFold Cross-validation

Now, a KLR model is evaluated using 10-Fold Cross-validation.

In [9]:
# Variables to used per alternative
variables = {1: ['cost', 'time'],
             2: ['cost', 'time'],
             3: ['cost', 'time'],
             4: ['cost', 'time']}

In [10]:
# Ignore warnings generated when fitting the model because the covariance matrix is singular.
import warnings
warnings.filterwarnings('ignore')

In [11]:
from sklearn.preprocessing import MinMaxScaler
    
fold = 0
precision = 0
recall = 0
fscore = 0

n_splits = 10

for train_set, test_set in pkl.KFold_cross_val(long_mode_choice,
                                               obs_id_column,
                                               n_splits,
                                               shuffle=False,
                                               random_state=1):
    
    fold += 1
    print("--- Set %d ---" % fold)
    
    # Scale the data
    scaler = MinMaxScaler()
    scaler.fit(train_set[['cost', 'time']])

    train_set_scaled = train_set.copy()
    train_set_scaled[['cost', 'time']] = scaler.transform(train_set[['cost', 'time']])

    test_set_scaled = test_set.copy()
    test_set_scaled[['cost', 'time']] = scaler.transform(test_set[['cost', 'time']])
    
    # Generate the train and test Kernel Matrices
    K_long_format_train = pkl.long_format_with_kernel_matrix(train_set_scaled, 
                                                             custom_alt_id, 
                                                             obs_id_column, 
                                                             choice_column, 
                                                             variables, 
                                                             kernel_type = "RBF")
    
    K_long_format_test = pkl.long_format_with_kernel_matrix(test_set_scaled, 
                                                            custom_alt_id, 
                                                            obs_id_column, 
                                                            choice_column, 
                                                            variables, 
                                                            kernel_type = "RBF", 
                                                            Z = train_set_scaled,
                                                            length_scale = 1)

    # Define the kernel specification
    [basic_specification, basic_names, total_vars] = pkl.define_kernel_specification(K_long_format_train, 
                                                                                     custom_alt_id,
                                                                                     obs_id_column,
                                                                                     specification=OrderedDict(),
                                                                                     names=OrderedDict(),
                                                                                     alpha_per_alternative=False,
                                                                                     intercept=1)

    
    # Estimate the multinomial logit model (MNL)
    kernel_model = pkl.create_choice_model(data=K_long_format_train,
                                           alt_id_col=custom_alt_id,
                                           obs_id_col=obs_id_column,
                                           choice_col=choice_column,
                                           specification=basic_specification,
                                           model_type="MNL",
                                           names=basic_names)
    
    # Specify the initial values and method for the optimization.
    kernel_model.fit_mle(np.zeros(total_vars), method="L-BFGS-B", PMLE="RIDGE", PMLE_lambda=4)
    
    (precision_fold, recall_fold, fscore_fold) = kernel_model.precision_recall_fscore(K_long_format_test, "micro", beta = 1)

    print("Precision: %f, Recall: %f, FScore: %f" % (precision_fold, recall_fold, fscore_fold))
    
    precision += precision_fold
    recall += recall_fold
    fscore += fscore_fold
    
print("\n --------------------- Final Results ---------------------")
print("Precision: %f, Recall: %f, FScore: %f" % (precision/n_splits, recall/n_splits, fscore/n_splits))

--- Set 1 ---
Log-likelihood at zero: -564.2218
Initial Log-likelihood: -564.2218
Estimation Time for Point Estimation: 0.06 seconds.
Final log-likelihood: -346.7395
Precision: 0.673913, Recall: 0.673913, FScore: 0.673913
--- Set 2 ---
Log-likelihood at zero: -564.2218
Initial Log-likelihood: -564.2218
Estimation Time for Point Estimation: 0.06 seconds.
Final log-likelihood: -343.2334
Precision: 0.586957, Recall: 0.586957, FScore: 0.586957
--- Set 3 ---
Log-likelihood at zero: -564.2218
Initial Log-likelihood: -564.2218
Estimation Time for Point Estimation: 0.05 seconds.
Final log-likelihood: -348.7580
Precision: 0.760870, Recall: 0.760870, FScore: 0.760870
--- Set 4 ---
Log-likelihood at zero: -565.6081
Initial Log-likelihood: -565.6081
Estimation Time for Point Estimation: 0.07 seconds.
Final log-likelihood: -350.5128
Precision: 0.733333, Recall: 0.733333, FScore: 0.733333
--- Set 5 ---
Log-likelihood at zero: -565.6081
Initial Log-likelihood: -565.6081
Estimation Time for Point Esti

## About this Jupyter notebook

This Jupyter notebook was developed by José Ángel Martín Baos on September 2019 to demonstrate the key functionalities of the PyKernelLogit Python package (https://pypi.org/project/pykernellogit/). This package is based on the PyLogit Python package (https://pypi.org/project/pylogit/) developed by Timothy Brathwaite (Timothy Brathwaite and Joan L. Walker. Asymmetric, closed-form, finite-parameter models of multinomial choice. Journal of Choice Modelling. Volume 29, December 2018.). This notebook is based on the example notebook for the PyLogit package.

* José Ángel Martín Baos (https://joseangelmartinb.github.io/). Faculty of Computer Science. University of Castilla-La Mancha. 
